In [1]:
import xlrd 
import pandas
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import xlwt

# import data from excel 
if __name__ == '__main__' :
    book = xlrd.open_workbook ('C:/research/python/data/Bric_seq_160119.xlsx')
    sheet1 = book.sheet_by_name('PUM1')
    data1 = []
    data1 = [[sheet1.cell_value(r, col)
                for col in range(sheet1.ncols)]
                    for r in range(sheet1.nrows)]

# numbers are registered as numbers, while leaving the strings alone
for i in range (len (data1)):
    for j in range(len(data1[0])):
        try:
            data1[i][j] = float (data1[i][j])
        except:
            pass

# Wrap the data in a pandas DataFrame
df = DataFrame(data1[1:], columns = data1[0])

if __name__ == "__main__":
    book = xlwt.Workbook()
    sheet = book.add_sheet("RNAseq_motif_vs_q", cell_overwrite_ok=True)
    tag = ['q', 'PUM_motif_up', 'fraction_up', 'PUM_motif_down', 'fraction_down' ]
    for j, ele in enumerate(tag):
        sheet.write(0, j, ele)
    
    for i in range(1, 21):
        q = 0.05 * i
        df_q = 'df(' + str(q) + ')'
    
        # extract the genes of which q_value is lower than q    
        df_q = df[df['q_value'] <= q]
    
        # define the genes which increases (df_up) or decrease (_down) by siPUM 
        df_up = df_q[df_q['log2(fold_change)'] >0]
        df_down = df_q[df_q['log2(fold_change)'] <0]
    
        # count the number of genes containing PUM motif
        df_up_motif = df_up[df_up['PUM_motif'] >0]
        df_down_motif = df_down[df_down['PUM_motif'] >0]
    
        #estimate the motif fraction
        percent_up = 1.0 * df_up_motif['PUM_motif'].count() / df_up['PUM_motif'].count()
        percent_down = 1.0 * df_down_motif['PUM_motif'].count() / df_down['PUM_motif'].count()
        
        sheet.write(i, 0, q)        
        sheet.write(i, 1, df_up['PUM_motif'].count())    
        sheet.write(i, 2, percent_up)
        sheet.write(i, 3, df_down['PUM_motif'].count())    
        sheet.write(i, 4, percent_down)
        
book.save("C:/research/python/data/motif_enrichment.xls")
